In [1]:
import tkinter
import tkinter as tk
from tkinter import *
from PIL import Image,ImageTk
import random
from tkinter import messagebox

main=tk.Tk()
main.title("GAMING ZONE")
main.geometry('900x600')
main.resizable(width=False,height=False)
main.config(bg='light grey')
title=Label(main,text="GAMING ZONE",font="Helvetica 30 bold italic",width=90,height=1,bg='light blue',fg='black',relief=RAISED).pack(side='top')

logo_label=Image.open('gaming logo.png')
logo_var=logo_label.resize((100,40))
logo=ImageTk.PhotoImage(logo_var)
logo_t=Label(main,image = logo,bg='light blue').place(x=0,y=0)

img_label=Image.open('side logo.png')
img_var=img_label.resize((350,450))
img=ImageTk.PhotoImage(img_var)
img_t=Label(main,image = img,bg='light grey').pack(side='right')


#log=Button(main,image = logo,bg='light grey').place(x=0,y=10)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#tic tac toe game
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


class TicTacToe:
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Tic Tac Toe")
        
        self.current_player = "X"
        self.board = [["" for _ in range(3)] for _ in range(3)]
        self.buttons = []
        
        for i in range(3):
            row = []
            for j in range(3):
                button = tk.Button(self.window, text="", width=20, height=10,
                                   command=lambda i=i, j=j: self.make_move(i, j))
                button.grid(row=i, column=j)
                row.append(button)
            self.buttons.append(row)
        
    def make_move(self, row, col):
        if self.board[row][col] == "":
            self.board[row][col] = self.current_player
            self.buttons[row][col].config(text=self.current_player)
            
            if self.check_win(row, col):
                messagebox.showinfo("Game Over", f"Player {self.current_player} wins!")
                self.reset_game()
            elif self.check_draw():
                messagebox.showinfo("Game Over", "It's a draw!")
                self.reset_game()
            else:
                self.switch_player()
    
    def switch_player(self):
        if self.current_player == "X":
            self.current_player = "O"
        else:
            self.current_player = "X"
    
    def check_win(self, row, col):
        symbol = self.board[row][col]
        # Check row
        if self.board[row][0] == self.board[row][1] == self.board[row][2] == symbol:
            return True
        # Check column
        if self.board[0][col] == self.board[1][col] == self.board[2][col] == symbol:
            return True
        # Check diagonals
        if self.board[0][0] == self.board[1][1] == self.board[2][2] == symbol:
            return True
        if self.board[0][2] == self.board[1][1] == self.board[2][0] == symbol:
            return True
        return False
    
    def check_draw(self):
        for row in self.board:
            if "" in row:
                return False
        return True
    
    def reset_game(self):
        self.board = [["" for _ in range(3)] for _ in range(3)]
        for row in self.buttons:
            for button in row:
                button.config(text="",width=20,height=10)
    
    def run(self):
        self.window.mainloop()

def tic():
    game = TicTacToe()
    game.run()

#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#tic toc game frame
t_frame=Label(main,width=16,height=8,bg='black',relief=RAISED,bd=5).place(x=50,y=110)
t_label=Image.open('tic.png')
t_var=t_label.resize((104,104))
B=ImageTk.PhotoImage(t_var)
t=Button(main,image = B,bg='light grey',activebackground='green',command=tic).place(x=57,y=120)
game_title=Label(main,text="Tic Toc Toe",font="Helvetica 10 bold italic",width=15,height=1,bg='light blue',fg='black',relief=RAISED).place(x=50,y=242)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#break brick game
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
class GameObject(object):
    def __init__(self, canvas, item):
        self.canvas = canvas
        self.item = item

    def get_position(self):
        return self.canvas.coords(self.item)

    def move(self, x, y):
        self.canvas.move(self.item, x, y)

    def delete(self):
        self.canvas.delete(self.item)


class Ball(GameObject):
    def __init__(self, canvas, x, y):
        self.radius = 10
        self.direction = [1, -1]
        # increase the below value to increase the speed of ball
        self.speed = 5
        item = canvas.create_oval(x-self.radius, y-self.radius,
                                  x+self.radius, y+self.radius,
                                  fill='white')
        super(Ball, self).__init__(canvas, item)

    def update(self):
        coords = self.get_position()
        width = self.canvas.winfo_width()
        if coords[0] <= 0 or coords[2] >= width:
            self.direction[0] *= -1
        if coords[1] <= 0:
            self.direction[1] *= -1
        x = self.direction[0] * self.speed
        y = self.direction[1] * self.speed
        self.move(x, y)

    def collide(self, game_objects):
        coords = self.get_position()
        x = (coords[0] + coords[2]) * 0.5
        if len(game_objects) > 1:
            self.direction[1] *= -1
        elif len(game_objects) == 1:
            game_object = game_objects[0]
            coords = game_object.get_position()
            if x > coords[2]:
                self.direction[0] = 1
            elif x < coords[0]:
                self.direction[0] = -1
            else:
                self.direction[1] *= -1

        for game_object in game_objects:
            if isinstance(game_object, Brick):
                game_object.hit()


class Paddle(GameObject):
    def __init__(self, canvas, x, y):
        self.width = 80
        self.height = 10
        self.ball = None
        item = canvas.create_rectangle(x - self.width / 2,
                                       y - self.height / 2,
                                       x + self.width / 2,
                                       y + self.height / 2,
                                       fill='#FFB643')
        super(Paddle, self).__init__(canvas, item)

    def set_ball(self, ball):
        self.ball = ball

    def move(self, offset):
        coords = self.get_position()
        width = self.canvas.winfo_width()
        if coords[0] + offset >= 0 and coords[2] + offset <= width:
            super(Paddle, self).move(offset, 0)
            if self.ball is not None:
                self.ball.move(offset, 0)


class Brick(GameObject):
    COLORS = {1: '#4535AA', 2: '#ED639E', 3: '#8FE1A2'}

    def __init__(self, canvas, x, y, hits):
        self.width = 75
        self.height = 20
        self.hits = hits
        color = Brick.COLORS[hits]
        item = canvas.create_rectangle(x - self.width / 2,
                                       y - self.height / 2,
                                       x + self.width / 2,
                                       y + self.height / 2,
                                       fill=color, tags='brick')
        super(Brick, self).__init__(canvas, item)

    def hit(self):
        self.hits -= 1
        if self.hits == 0:
            self.delete()
        else:
            self.canvas.itemconfig(self.item,
                                   fill=Brick.COLORS[self.hits])


class Game(tk.Frame):
    def __init__(self, master):
        super(Game, self).__init__(master)
        self.lives = 3
        self.width = 610
        self.height = 400
        self.canvas = tk.Canvas(self, bg='#D6D1F5',
                                width=self.width,
                                height=self.height,)
        self.canvas.pack()
        self.pack()

        self.items = {}
        self.ball = None
        self.paddle = Paddle(self.canvas, self.width/2, 326)
        self.items[self.paddle.item] = self.paddle
        # adding brick with different hit capacities - 3,2 and 1
        for x in range(5, self.width - 5, 75):
            self.add_brick(x + 37.5, 50, 3)
            self.add_brick(x + 37.5, 70, 2)
            self.add_brick(x + 37.5, 90, 1)

        self.hud = None
        self.setup_game()
        self.canvas.focus_set()
        self.canvas.bind('<Left>',
                         lambda _: self.paddle.move(-10))
        self.canvas.bind('<Right>',
                         lambda _: self.paddle.move(10))

    def setup_game(self):
           self.add_ball()
           self.update_lives_text()
           self.text = self.draw_text(300, 200,
                                      'Press Space to start')
           self.canvas.bind('<space>', lambda _: self.start_game())

    def add_ball(self):
        if self.ball is not None:
            self.ball.delete()
        paddle_coords = self.paddle.get_position()
        x = (paddle_coords[0] + paddle_coords[2]) * 0.5
        self.ball = Ball(self.canvas, x, 310)
        self.paddle.set_ball(self.ball)

    def add_brick(self, x, y, hits):
        brick = Brick(self.canvas, x, y, hits)
        self.items[brick.item] = brick

    def draw_text(self, x, y, text, size='40'):
        font = ('Forte', size)
        return self.canvas.create_text(x, y, text=text, font=font)

    def update_lives_text(self):
        text = 'Lives: %s' % self.lives
        if self.hud is None:
            self.hud = self.draw_text(50, 20, text, 15)
        else:
            self.canvas.itemconfig(self.hud, text=text)

    def start_game(self):
        self.canvas.unbind('<space>')
        self.canvas.delete(self.text)
        self.paddle.ball = None
        self.game_loop()

    def game_loop(self):
        self.check_collisions()
        num_bricks = len(self.canvas.find_withtag('brick'))
        if num_bricks == 0: 
            self.ball.speed = None
            self.draw_text(300, 200, 'You win! You the Breaker of Bricks.')
        elif self.ball.get_position()[3] >= self.height: 
            self.ball.speed = None
            self.lives -= 1
            if self.lives < 0:
                self.draw_text(300, 200, 'You Lose! Game Over!')
            else:
                self.after(1000, self.setup_game)
        else:
            self.ball.update()
            self.after(50, self.game_loop)

    def check_collisions(self):
        ball_coords = self.ball.get_position()
        items = self.canvas.find_overlapping(*ball_coords)
        objects = [self.items[x] for x in items if x in self.items]
        self.ball.collide(objects)


def brickgame ():
    root = tk.Tk()
    root.title('Break those Bricks!')    
    game = Game(root)
    root.geometry('900x600')
    root.resizable(width=False,height=False)
    root.config(bg='light grey')
    title=Label(root,text="Break The Bricks",font="Helvetica 30 bold italic",width=90,height=1,bg='light blue',fg='black',relief=RAISED).pack(side='bottom')
    back=Button(root,text="Back",font="Helvetica 20 bold italic",bg='red',relief=RAISED,command=root.destroy).pack(side='bottom')
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------    
#break brick game frame
bb_frame=Label(main,width=16,height=8,bg='black',relief=RAISED,bd=5).place(x=240,y=110)
bb_label=Image.open('break and brick.png')
bb_var=bb_label.resize((104,104))
bb_i=ImageTk.PhotoImage(bb_var)
bb=Button(main,image = bb_i,bg='light grey',activebackground='green',command=brickgame).place(x=247,y=120)
b_title=Label(main,text="Break-Brick",font="Helvetica 10 bold italic",width=15,height=1,bg='light blue',fg='black',relief=RAISED).place(x=240,y=242)
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#sudoku game
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def suk():
    global grid,entry_list
    guess = str()
    grid = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]

    root = Tk()
    root.title("Sudoku")
    root.minsize(width=500, height=635)
    root.attributes("-alpha", 0.90)
    root.configure(background="#282828")
    root.geometry('900x600')
    root.resizable(width=False,height=False)
    root.config(bg='light grey')

    title = Label(root, text='Sudoku', font="Geneva 30 bold italic",width=90,height=1,bg='light blue',fg='black',relief=RAISED).pack(side='top')

    box = Canvas(root, width = 435, height = 435,background="#282828",bd=6, highlightthickness=6)  
    box.pack()

    entry_list = [[],[],[]]
    var =[]
    done = False

    def one_grid(row):
        global grid,entry_list
        g1 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        entry_list[0].append(g1)
        g1.place(x = 5,y = 5)
        g2 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        entry_list[0].append(g2)
        g2.place(x = 52,y = 5)
        g3 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        g3.place(x = 102,y = 5)
        entry_list[0].append(g3)
        g4 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        g4.place(x = 5,y = 52)
        entry_list[1].append(g4)
        g5 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        g5.place(x = 52,y = 52)
        entry_list[1].append(g5)
        g6 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        g6.place(x = 102,y = 52)
        entry_list[1].append(g6)
        g7 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        g7.place(x = 5,y = 102)
        entry_list[2].append(g7)
        g8 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        g8.place(x = 52,y = 102)
        entry_list[2].append(g8)
        g9 = Entry(row,textvariable=var,width = 2, fg="white", font="Geneva 30 bold",bg="#373737",justify=CENTER)
        g9.place(x = 102,y = 102)
        entry_list[2].append(g9)

    def display_val():
        global entry_list
        u=0
        for a in entry_list:
            a_splited = [a[x:x+9] for x in range(0, len(a), 9)]
            for y in range(9):
                if(grid[u][y] != 0):
                    a_splited[0][y].insert(0,grid[u][y]) 
            u+=1
        u=3
        for a in entry_list:
            a_splited = [a[x:x+9] for x in range(0, len(a), 9)]
            for y in range(9):
                if(grid[u][y] != 0):
                    a_splited[1][y].insert(0,grid[u][y]) 
            u+=1

        u=6
        for a in entry_list:
            a_splited = [a[x:x+9] for x in range(0, len(a), 9)]
            for y in range(9):
                if(grid[u][y] != 0):
                    a_splited[2][y].insert(0,grid[u][y]) 
            u+=1



    def clear():
        global grid
        grid = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]

    def scramble():
        global grid
        clear()
        for a in entry_list:
            for b in a:
                b.delete(first=0,last=100)
        amount = 20


        for i in range(amount):
            y = random.randint(0,len(grid)-1)
            x = random.randint(0,len(grid)-1)
            num = random.randint(1,len(grid))
            allow = 0
            for e in range(len(grid)):
                if num not in grid[x] and num != grid[e][y]:
                    allow +=1
            grid[x][y] = num
            tempo = grid     
            tempo = rearrange(tempo)

            for e in range(len(grid)):
                if(duplicate_checker(tempo[e])):
                    allow = 0
            if allow !=len(grid):
                grid[x][y] = 0

        display_val()


    def stay(num,x,y):

        for e in range(9):
            if grid[y][e] ==num:
                return False
        for e in range(9):
            if grid[e][x] ==num:
                return False

        for i in range(3):
            for e in range(3):
                if grid[((y//3)*3)+i][((x//3)*3)+e] ==num:
                    return False
        return True

    def pressed_solve():
        global grid,done
        done =False
        solver()
    def solver():
        global grid,done

        if(done ==False):

            for y in range(9):
                for x in range(9):
                    if grid[y][x] == 0:
                        for num in range(1,10):
                            if stay(num,x,y):
                                grid[y][x]=num
                                solver()
                                grid[y][x]=0
                        return
            done =True
            for a in entry_list:
                for b in a:
                    b.delete(first=0,last=100)
            display_val()

            return

    def rearrange(a):
        temp=[[],[],[],[],[],[],[],[],[]]
        count =0
        ch = 0
        for e in range(len(a)):
            for x in range(3):
                if(a[e][x]!=0):
                    temp[ch].append(a[e][x])
            count+=1
            if(count ==3):
                ch+=1
                count = 0
        for e in range(len(a)):
            for x in range(3,6):
                if(a[e][x]!=0):
                    temp[ch].append(a[e][x])
            count+=1
            if(count ==3):
                ch+=1
                count = 0
        for e in range(len(a)):
            for x in range(6,9):
                if(a[e][x]!=0):
                    temp[ch].append(a[e][x])
            count+=1
            if(count ==3):
                ch+=1
                count = 0
        return temp

    def duplicate_checker(a):
            b = set(a)
            result = len(a) != len(b)
            #print(result)
            if(result == True):
                return True

    def checkrow_horz(a):
        for x in a:
            if(duplicate_checker(x) == True):
                return True


    def checkrow_vert(a):
        for y in range(len(a)):
            temp = []
            for x in a:
                temp.append(x[y])
            if(duplicate_checker(temp) == True):
                return True

    def checkcol(a):
        for y in range(3):
            temp = []
            for x in range(int(len(a)/3)):
                temp.append(a[x][3*y])
                temp.append(a[x][3*y+1])
                temp.append(a[x][3*y+2])
            if(duplicate_checker(temp) == True):
               return True
            temp = []
            for x in range(3,(int(len(a)/3))*2):
                temp.append(a[x][3*y])
                temp.append(a[x][3*y+1])
                temp.append(a[x][3*y+2])
            if(duplicate_checker(temp) == True):
               return True
            temp = []
            for x in range(6,(int(len(a)/3))*3):
                temp.append(a[x][3*y])
                temp.append(a[x][3*y+1])
                temp.append(a[x][3*y+2])
            if(duplicate_checker(temp) == True):
               return True

    def submit():
        global entry_list

        temp = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
        u=0
        for a in entry_list:
            a_splited = [a[x:x+9] for x in range(0, len(a), 9)]
            for y in range(9):
                if(a_splited[0][y].get() != '' ):
                    temp[u][y]= int(a_splited[0][y].get())
            u+=1
        u=3
        for a in entry_list:
            a_splited = [a[x:x+9] for x in range(0, len(a), 9)]
            for y in range(9):
                if(a_splited[1][y].get() != '' ):
                    temp[u][y]= int(a_splited[1][y].get())
            u+=1

        u=6
        for a in entry_list:
            a_splited = [a[x:x+9] for x in range(0, len(a), 9)]
            for y in range(9):
                if(a_splited[2][y].get() != '' ):
                    temp[u][y]= int(a_splited[2][y].get())
            u+=1

        if(checkrow_horz(temp) == True or checkrow_vert(temp) ==True or checkcol(temp) ==True):
            wrong()
        else:
            correct()

    def wrong():
        title.config(fg="#990000",text="Try Again")
        row1.config(highlightbackground="#990000")
        row2.config(highlightbackground="#990000")
        row3.config(highlightbackground="#990000")
        row4.config(highlightbackground="#990000")
        row5.config(highlightbackground="#990000")
        row6.config(highlightbackground="#990000")
        row7.config(highlightbackground="#990000")
        row8.config(highlightbackground="#990000")
        row9.config(highlightbackground="#990000")
        row1.after(2100, lambda: row1.config(highlightbackground="white"))
        row2.after(2100, lambda: row2.config(highlightbackground="white"))
        row3.after(2100, lambda: row3.config(highlightbackground="white"))
        row4.after(2100, lambda: row4.config(highlightbackground="white"))
        row5.after(2100, lambda: row5.config(highlightbackground="white"))
        row6.after(2100, lambda: row6.config(highlightbackground="white"))
        row7.after(2100, lambda: row7.config(highlightbackground="white"))
        row8.after(2100, lambda: row8.config(highlightbackground="white"))
        row9.after(2100, lambda: row9.config(highlightbackground="white"))
        title.after(2100, lambda: title.config(fg="#382888",text ="Sudoku"))

    def correct():
        title.config(fg="#288888",text="Correct")
        row1.config(highlightbackground="#288888")
        row2.config(highlightbackground="#288888")
        row3.config(highlightbackground="#288888")
        row4.config(highlightbackground="#288888")
        row5.config(highlightbackground="#288888")
        row6.config(highlightbackground="#288888")
        row7.config(highlightbackground="#288888")
        row8.config(highlightbackground="#288888")
        row9.config(highlightbackground="#288888")
        row1.after(2100, lambda: row1.config(highlightbackground="white"))
        row2.after(2100, lambda: row2.config(highlightbackground="white"))
        row3.after(2100, lambda: row3.config(highlightbackground="white"))
        row4.after(2100, lambda: row4.config(highlightbackground="white"))
        row5.after(2100, lambda: row5.config(highlightbackground="white"))
        row6.after(2100, lambda: row6.config(highlightbackground="white"))
        row7.after(2100, lambda: row7.config(highlightbackground="white"))
        row8.after(2100, lambda: row8.config(highlightbackground="white"))
        row9.after(2100, lambda: row9.config(highlightbackground="white"))
        title.after(2100, lambda: title.config(fg="#382888",text ="Sudoku"))


    row1 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row1)
    row1.place(x = 0,y = 0)
    row2 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row2)
    row2.place(x = 150,y = 0)
    row3 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row3)
    row3.place(x = 300,y = 0)

    row4 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row4)
    row4.place(x = 0,y = 150)
    row5 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row5)
    row5.place(x = 150,y = 150)
    row6 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row6)
    row6.place(x = 300,y = 150)

    row7 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row7)
    row7.place(x = 0,y = 300)
    row8 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row8)
    row8.place(x = 150,y = 300)
    row9 = Canvas(box, width = 120, height = 120,background="#282828",bd=10, highlightthickness=10)
    one_grid(row9)
    row9.place(x = 300,y = 300)

    scramble()


    #but = Canvas(root, width = 500, height = 45,background="#282828",bd=13, highlightthickness=0)  
    #but.pack()
    enter = Button(root,text ="Submit",command=submit, font="Geneva 30 bold",bg='grey',highlightbackground="red",activebackground='#382888',disabledforeground="#382888")
    #enter.configure(bg='grey')
    enter.place(x = 20,y = 530)
    reset = Button(root,text ="Reset", fg="#282828",command=scramble, font="Geneva 30 bold",bg='#382888',activebackground='#382888',highlightbackground="#382888",disabledforeground="#382888",justify=CENTER)
    #reset.configure(bg='#382888')
    reset.place(x = 470,y = 530)
    solve = Button(root,text ="Solve", fg="#282828",command=pressed_solve, font="Geneva 30 bold",bg='#FCD4D4',activebackground='#FCD4D4',highlightbackground="#FCD4D4",disabledforeground="#FCD4D4",justify=CENTER)
    solve.configure(bg='#FCD4D4')
    solve.place(x=270,y=530)
    
    def exit():
        root.destroy()
    exit_button=Button(root,text="EXIT",fg="#282828",command=exit, font="Geneva 30 bold",bg='red',activebackground='#FCD4D4',highlightbackground="#FCD4D4",disabledforeground="#FCD4D4",justify=CENTER)
    exit_button.place(x=700,y=530)
    # exit button


    root.mainloop()

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#sudoku frame
su_frame=Label(main,width=16,height=8,bg='black',relief=RAISED,bd=5).place(x=430,y=110)
su_label=Image.open('sudoku.png')
su_var=su_label.resize((104,104))
su_i=ImageTk.PhotoImage(su_var)
su=Button(main,image = su_i,bg='light grey',command=suk,activebackground='green').place(x=437,y=120)
s_title=Label(main,text="Sudoku",font="Helvetica 10 bold italic",width=15,height=1,bg='light blue',fg='black',relief=RAISED).place(x=430,y=242)
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#snake game
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
def snak():
    global direction, score  # Declare direction and score as global variables

    # Initialising Dimensions of Game
    WIDTH = 500
    HEIGHT = 500
    SPEED = 200
    SPACE_SIZE = 20
    BODY_SIZE = 2
    SNAKE = "#00FF00"
    FOOD = "#FFFFFF"
    BACKGROUND = "#000000"

    # Class to design the snake
    class Snake:
        def __init__(self, window):
            self.window = window
            self.body_size = BODY_SIZE
            self.coordinates = []
            self.squares = []

            for i in range(0, BODY_SIZE):
                self.coordinates.append([0, 0])

            for x, y in self.coordinates:
                square = canvas.create_rectangle(x, y, x + SPACE_SIZE, y + SPACE_SIZE, fill=SNAKE, tag="snake")
                self.squares.append(square)

    # Class to design the food
    class Food:
        def __init__(self, window):
            self.window = window
            x = random.randint(0, (WIDTH / SPACE_SIZE) - 1) * SPACE_SIZE
            y = random.randint(0, (HEIGHT / SPACE_SIZE) - 1) * SPACE_SIZE

            self.coordinates = [x, y]

            canvas.create_oval(x, y, x + SPACE_SIZE, y + SPACE_SIZE, fill=FOOD, tag="food")

    # Function to check the next move of snake
    def next_turn(snake, food):
        global direction, score

        x, y = snake.coordinates[0]

        if direction == "up":
            y -= SPACE_SIZE
        elif direction == "down":
            y += SPACE_SIZE
        elif direction == "left":
            x -= SPACE_SIZE
        elif direction == "right":
            x += SPACE_SIZE

        snake.coordinates.insert(0, (x, y))

        square = canvas.create_rectangle(x, y, x + SPACE_SIZE, y + SPACE_SIZE, fill=SNAKE)

        snake.squares.insert(0, square)

        if x == food.coordinates[0] and y == food.coordinates[1]:
            score += 1
            label.config(text="Points:{}".format(score))
            canvas.delete("food")
            food = Food(window)
        else:
            del snake.coordinates[-1]
            canvas.delete(snake.squares[-1])
            del snake.squares[-1]

        if check_collisions(snake):
            game_over()
        else:
            window.after(SPEED, next_turn, snake, food)

    # Function to control direction of snake
    def change_direction(new_direction):
        global direction

        if new_direction == 'left':
            if direction != 'right':
                direction = new_direction
        elif new_direction == 'right':
            if direction != 'left':
                direction = new_direction
        elif new_direction == 'up':
            if direction != 'down':
                direction = new_direction
        elif new_direction == 'down':
            if direction != 'up':
                direction = new_direction

    # Function to check snake's collision and position
    def check_collisions(snake):
        x, y = snake.coordinates[0]

        if x < 0 or x >= WIDTH:
            return True
        elif y < 0 or y >= HEIGHT:
            return True

        for body_part in snake.coordinates[1:]:
            if x == body_part[0] and y == body_part[1]:
                return True

        return False

    # Function to display "GAME OVER" when the game ends
    def game_over():
        canvas.delete(ALL)
        canvas.create_text(canvas.winfo_width() / 2, canvas.winfo_height() / 2, font=('consolas', 70),
                           text="GAME OVER", fill="red", tag="gameover")

    # Create the main window
    window = Toplevel(main)
    window.title("GFG Snake game ")

    score = 0
    direction = 'down'

    # Display of Points Scored in Game
    label = Label(window, text="Points:{}".format(score), font=('consolas', 20))
    label.pack()

    canvas = Canvas(window, bg=BACKGROUND, height=HEIGHT, width=WIDTH)
    canvas.pack()

    window.update()

    window_width = window.winfo_width()
    window_height = window.winfo_height()
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()

    x = int((screen_width / 2) - (window_width / 2))
    y = int((screen_height / 2) - (window_height / 2))

    window.geometry(f"{window_width}x{window_height}+{x}+{y}")

    # Bind the arrow keys to the 'window' and set focus
    window.bind('<Left>', lambda event: change_direction('left'))
    window.bind('<Right>', lambda event: change_direction('right'))
    window.bind('<Up>', lambda event: change_direction('up'))
    window.bind('<Down>', lambda event: change_direction('down'))

    # Set focus to the 'window'
    window.focus_set()

    snake = Snake(window)
    food = Food(window)

    next_turn(snake, food)
#---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#snake frame
sn_frame=Label(main,width=16,height=8,bg='black',relief=RAISED,bd=5).place(x=150,y=330)
sn_label=Image.open('snake.png')
sn_var=sn_label.resize((104,104))
sn_i=ImageTk.PhotoImage(sn_var)
snake_window_btn=Button(main,image = sn_i,bg='light grey',command=snak,activebackground='green').place(x=157,y=340)
s_title=Label(main,text="Snake",font="Helvetica 10 bold italic",width=15,height=1,bg='light blue',fg='black',relief=RAISED).place(x=150,y=463)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#number chooes game
#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#secret number
def number():
    global secret_number, button_list
    root = Tk()
    root.title("MOO ICT")
    root.geometry("900x600")
    secret_number = "none"

    #---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    # function and events
    def StartGame():
        global secret_number
        for button in button_list:
            button.config(text=str(random.randint(0, 100)))

        randomButton = random.choice(button_list)
        secret_number = randomButton.cget("text")
        print("Secret Number is: ", secret_number)
    def OnClick(event):
        btn = event.widget
        buttonText = btn.cget("text")
        if secret_number == buttonText:
            answer_label.config(text="Yes it was " + secret_number)
            StartGame()
        else:
            answer_label.config(text="Rong number Try Again " + secret_number)
            StartGame()
    # gui components
    title_label = Label(root, text="Guess the Secret Number", font="Helvetica 30 bold italic",width=40,height=1,bg='light blue',fg='black',relief=RAISED, justify="center")
    button_one = Button(root, text="00", font=("Helvetical 15"), width=6, pady=15, bg="palegreen")
    button_two = Button(root, text="00", font=("Helvetical 15"), width=6, pady=15, bg="skyblue")
    button_three = Button(root, text="00", font=("Helvetical 15"), width=6, pady=15, bg="coral")
    button_list = [button_one, button_two, button_three]
    answer_label = Label(root, text="Answer", font=("Helvetical 15"), pady=9, fg="purple", justify="center")
    button_back = Button(root, text="Back", font=("Helvetical 15"), width=6, pady=15, bg="red",command=root.destroy)
    title_label.grid(row = 0, column=0,columnspan=3)
    button_one.grid(row=1, column=0)
    button_two.grid(row=1, column=1)
    button_three.grid(row=1, column=2)
    answer_label.grid(row = 3, column=0,columnspan=3)
    button_back.grid(row=4,column=0,columnspan=3)
    button_one.bind("<Button-1>", OnClick)
    button_two.bind("<Button-1>", OnClick)
    button_three.bind("<Button-1>", OnClick)
    StartGame()

    root.mainloop()

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#number chooes frame
no_frame=Label(main,width=16,height=8,bg='black',relief=RAISED,bd=5).place(x=340,y=330)
no_label=Image.open('number chooes.png')
no_var=no_label.resize((104,104))
no_i=ImageTk.PhotoImage(no_var)
no=Button(main,image = no_i,bg='light grey',command=number,activebackground='green').place(x=347,y=340)
n_title=Label(main,text="Number Guessing",font="Helvetica 10 bold italic",width=15,height=1,bg='light blue',fg='black',relief=RAISED).place(x=340,y=463)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# exit button
exit_button=Button(main,text="EXIT",font="Helvetica 20 bold italic",bg='red',width=10,relief=RAISED,command=main.destroy).place(x=380,y=530)
main.mainloop()